In [5]:
import os
from dotenv import load_dotenv
from groq import Groq

In [6]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
print(groq_api_key)

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


In [10]:
client = Groq(
    api_key=groq_api_key,
)

#note que o groq pega uma lista de mensagens, não uma única string
chat_completion = client.chat.completions.create(
    #primeiro parametro é a lista de mensagens
    messages=[

        {

            "role": "user",

            "content": "Explain the importance of fast language models",

        }

    ],
    #segundo parametro é o modelo de linguagem que será usado (é possível setar outro modelo que a API hospeda)
    model="llama-3.3-70b-versatile",
    
)


print(chat_completion.choices[0].message.content)


Fast language models are crucial in today's technology landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models enable rapid processing of large amounts of text data. This is particularly important in applications where speed is critical, such as real-time chatbots, voice assistants, and sentiment analysis. By quickly analyzing and generating text, these models can respond to user queries and provide accurate results in a timely manner.

2. **Scalability**: As the volume of text data increases, fast language models become even more essential. They allow developers to handle large datasets and perform tasks such as text classification, language translation, and named entity recognition at scale. This scalability is critical for applications that process vast amounts of text data, such as social media monitoring, content moderation, and document analysis.

3. **Low Latency**: Fast language models enable low-latency in

In [ ]:
#Agora vamos criar uma classe de Agente
class Agent:
    def __init__(self, client, system):
        self.client = client #o cliente do groq
        self.system = system #o system prompt
        self.messages = [] #lista de mensagens
        if self.messages is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            messages=self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content
